In [6]:
import pandas as pd
rate_df = pd.read_csv('./movie/user_ratedmovies.dat', delimiter='\t',encoding='ISO-8859-1')
rate_df.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30


In [7]:
rate_df['datetime'] = pd.to_datetime(rate_df[['date_year', 'date_month', 'date_day', 
                                    'date_hour', 'date_minute', 'date_second']].rename(columns={'date_year': 'year', 
                                                                                               'date_month': 'month', 
                                                                                               'date_day': 'day', 
                                                                                               'date_hour': 'hour', 
                                                                                               'date_minute': 'minute', 
                                                                                               'date_second': 'second'}))

sorted_df = rate_df.sort_values(by=['userID', 'datetime'])
sorted_df.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,datetime
25,75,2011,2.0,29,10,2006,23,16,39,2006-10-29 23:16:39
9,75,420,2.0,29,10,2006,23,16,42,2006-10-29 23:16:42
3,75,160,2.0,29,10,2006,23,16,52,2006-10-29 23:16:52
19,75,1304,2.5,29,10,2006,23,16,56,2006-10-29 23:16:56
8,75,353,3.5,29,10,2006,23,17,0,2006-10-29 23:17:00


In [8]:
sorted_df['date'] = sorted_df['datetime'].dt.date
grouped = sorted_df.groupby(['userID', 'date'])
list_of_lists = [group['movieID'].tolist() for _, group in grouped]

In [9]:
filtered_lists = [inner_list for inner_list in list_of_lists if 3 < len(inner_list)] #can take final

In [10]:
import random
all_movie_ids_set = set([movie_id for inner_list in filtered_lists for movie_id in inner_list])

movies_except_last = []
last_movie_ids = []
shuffled_movie_ids_with_last = []

for inner_list in filtered_lists:
    except_last = inner_list[:-1]
    movies_except_last.append(except_last)
    
    last_movie_id = inner_list[-1]
    last_movie_ids.append(last_movie_id)
    
    possible_ids = list(all_movie_ids_set.difference(except_last))
    random_ids = random.sample(possible_ids, 49)
    
    random_ids.append(last_movie_id)
    random.shuffle(random_ids)
    shuffled_movie_ids_with_last.append(random_ids)

In [11]:
import pickle
from sklearn.model_selection import train_test_split

zipped_lists = list(zip(movies_except_last, last_movie_ids, shuffled_movie_ids_with_last))
train_zipped, test_zipped = train_test_split(zipped_lists, test_size=0.2, random_state=42)

#You can dump train/test_zipped[10] for testing
with open('./movie/example_train.txt', 'wb') as file:  # Further split train into train and val when training!
    pickle.dump(train_zipped, file)
with open('./movie/example_test.txt', 'wb') as file:
    pickle.dump(test_zipped, file)


In [12]:
#dataset(title) for GPT
import pickle
import pandas as pd
movie_df = pd.read_csv('./movie/movies.dat', delimiter='\t',encoding='ISO-8859-1')
with open("./movie/example_train.txt", "rb") as file:
    traindata = pickle.load(file)
traindata = list(traindata)
with open("./movie/example_test.txt", "rb") as file:
    testdata = pickle.load(file)
testdata = list(testdata)

In [13]:
def get_text_data(traindata, movie_df, num):
    data_dicts = []
    for i in range(num):
        data_dict = {}
        prompt = "I watched the following movies in order: "
        label = movie_df.loc[movie_df['id'] == traindata[i][1], 'title'].values[0]

        for movieID in traindata[i][0]:
            title = movie_df.loc[movie_df['id'] == movieID, 'title'].values[0]
            prompt = prompt + title +"; "
        prompt = prompt[:-2] +'. Based on these interactions, recommend a movie for me to watch next from a candidate set: '
            
        for movieID in traindata[i][2]:
            title = movie_df.loc[movie_df['id'] == movieID, 'title'].values[0]
            prompt = prompt + title +"; "
            
        prompt = prompt[:-2] +'. Please recommend from the candidate set. List the 10 recommendations in numbered bullet points.'
        data_dict['prompt'] = prompt
        data_dict['label'] = label
        data_dicts.append(data_dict)           
    return data_dicts

In [14]:
import json
with open('./movie/example_train_text.json', 'w') as file:
    json.dump(get_text_data(traindata, movie_df, 100), file)
with open('./movie/example_test_text.json', 'w') as file:
    json.dump(get_text_data(testdata, movie_df, 100), file)
    
#till here, we finish data preparation